In [4]:
pip install pygad

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import xgboost as xgb
import joblib

# List of sector-wise dataset files
dataset_files = {
    "Agriculture": "agriculture_modified_data.csv",
    "Industrial": "industrial_modified_data.csv",
    "IT": "it_modified_data.csv",
    "Manufacturing": "manufacturing_modified_data.csv",
    "Residential": "residential_modified_data.csv",
    "Transport": "transport_modified_data.csv",
    "Power": "power_modified_data.csv"
}

best_models = {}

for sector, file_path in dataset_files.items():
    try:
        print(f"\nTraining for {sector} sector...")

        # Load dataset
        df = pd.read_csv(file_path)
        df.dropna(inplace=True)

        if "emissions_tons" not in df.columns:
            print(f"Error: 'emissions_tons' column missing in {sector} dataset.")
            continue

        # Define sector-specific features
        sector_features = [col for col in df.columns if col != "emissions_tons"]

        # Feature Scaling
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()
        X = scaler_X.fit_transform(df[sector_features])
        y = scaler_y.fit_transform(df["emissions_tons"].values.reshape(-1, 1)).flatten()

        # Save scalers
        joblib.dump(scaler_X, f"scaler_{sector}.pkl")
        joblib.dump(scaler_y, f"scaler_y_{sector}.pkl")

        # Split dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # LSTM Model
        X_train_lstm = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        lstm_model = Sequential([
            tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dense(1, activation="linear")
        ])
        lstm_model.compile(loss="mse", optimizer="adam")
        lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test), verbose=0)
        lstm_model.save(f"lstm_model_{sector}.h5")

        # XGBoost Model
        xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)
        xgb_model.fit(X_train, y_train)
        joblib.dump(xgb_model, f"xgb_model_{sector}.pkl")

        # Evaluate Model
        y_pred = xgb_model.predict(X_test)
        mse = np.mean((y_pred - y_test) ** 2)

        print(f"MSE for {sector}: {mse:.6f}")
        best_models[sector] = {"XGBoost": xgb_model, "LSTM": lstm_model, "MSE": mse}

    except Exception as e:
        print(f"Error processing {sector}: {e}")

# Final Results
print("\nFinal Model Performance Per Sector:")
for sector, model_info in best_models.items():
    print(f"{sector}: MSE = {model_info['MSE']:.6f}")



Training for Agriculture sector...


MSE for Agriculture: 0.002068

Training for Industrial sector...


MSE for Industrial: 0.001551

Training for IT sector...


MSE for IT: 0.001638

Training for Manufacturing sector...


MSE for Manufacturing: 0.001762

Training for Residential sector...


MSE for Residential: 0.001370

Training for Transport sector...


MSE for Transport: 0.002250

Training for Power sector...


MSE for Power: 0.001385

Final Model Performance Per Sector:
Agriculture: MSE = 0.002068
Industrial: MSE = 0.001551
IT: MSE = 0.001638
Manufacturing: MSE = 0.001762
Residential: MSE = 0.001370
Transport: MSE = 0.002250
Power: MSE = 0.001385


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

# List of sector-wise dataset files
dataset_files = {
    "Agriculture": "agriculture_modified_data.csv",
    "Industrial": "industrial_modified_data.csv",
    "IT": "it_modified_data.csv",
    "Manufacturing": "manufacturing_modified_data.csv",
    "Residential": "residential_modified_data.csv",
    "Transport": "transport_modified_data.csv",
    "Power": "power_modified_data.csv"
}

for sector, file_path in dataset_files.items():
    try:
        print(f"\nProcessing {sector} sector...")

        # Load dataset
        df = pd.read_csv(file_path)

        # Drop non-numeric columns
        drop_columns = ["id", "timestamp", "user_type", "country", "sector"]
        df.drop(columns=[col for col in drop_columns if col in df.columns], errors='ignore', inplace=True)

        if "emissions_tons" not in df.columns:
            print(f"Error processing {sector}: 'emissions_tons' column not found.")
            continue

        # Fill missing values
        df.fillna(df.mean(), inplace=True)

        # One-hot encoding for categorical variables
        df = pd.get_dummies(df, drop_first=True)

        # Feature Scaling
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        features = [col for col in df.columns if col != "emissions_tons"]
        X = scaler_X.fit_transform(df[features])
        y = scaler_y.fit_transform(df["emissions_tons"].values.reshape(-1, 1)).flatten()

        # Split dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train XGBoost Model
        xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)
        xgb_model.fit(X_train, y_train)

        # Print Expected Features
        print(f"Expected features for {sector}: {features}")

    except Exception as e:
        print(f"Error processing {sector}: {e}")



Processing Agriculture sector...
Expected features for Agriculture: ['fertilizer_use', 'water_consumption', 'machinery_hours', 'land_area']

Processing Industrial sector...
Expected features for Industrial: ['energy_use', 'production_volume', 'waste_generated', 'operating_hours']

Processing IT sector...
Expected features for IT: ['server_count', 'energy_use', 'data_transferred', 'cooling_load']

Processing Manufacturing sector...
Expected features for Manufacturing: ['raw_material_input', 'machine_runtime', 'output_quantity', 'energy_consumption']

Processing Residential sector...
Expected features for Residential: ['electricity_usage', 'gas_consumption', 'household_size', 'appliance_count']

Processing Transport sector...
Expected features for Transport: ['fuel_consumption', 'vehicle_count', 'distance_traveled', 'load_capacity']

Processing Power sector...
Expected features for Power: ['coal_used', 'gas_used', 'renewable_input', 'total_output']


In [7]:
import shap
import joblib
import pandas as pd
import numpy as np
import xgboost as xgb
import os

# List of sector-wise dataset files
dataset_files = {
    "Agriculture": "agriculture_modified_data.csv",
    "Industrial": "industrial_modified_data.csv",
    "IT": "it_modified_data.csv",
    "Manufacturing": "manufacturing_modified_data.csv",
    "Residential": "residential_modified_data.csv",
    "Transport": "transport_modified_data.csv",
    "Power": "power_modified_data.csv"
}

shap_values_dict = {}

for sector, file_path in dataset_files.items():
    try:
        print(f"\nGenerating SHAP explainability for {sector} sector...")

        # Load dataset
        df = pd.read_csv(file_path)
        df.dropna(inplace=True)

        if "emissions_tons" not in df.columns:
            print(f"Error: 'emissions_tons' column missing in {sector} dataset.")
            continue

        # Load trained XGBoost model
        model_filename = f"xgb_model_{sector}.pkl"
        if not os.path.exists(model_filename):
            print(f"Trained model not found for {sector}, skipping SHAP analysis.")
            continue

        xgb_model = joblib.load(model_filename)

        # Load the feature scaler
        scaler_X = joblib.load(f"scaler_{sector}.pkl")

        # Extract sector-specific features
        sector_features = [col for col in df.columns if col != "emissions_tons"]
        X = scaler_X.transform(df[sector_features])

        # SHAP Explainer
        explainer = shap.Explainer(xgb_model)
        shap_values = explainer(X)

        # Save SHAP values
        shap_file = f"shap_values_{sector}.pkl"
        joblib.dump(shap_values, shap_file)
        shap_values_dict[sector] = shap_values

        print(f"SHAP values saved for {sector} in {shap_file}")

    except Exception as e:
        print(f"Error processing {sector}: {e}")

# Final Message
print("\nSHAP explainability generated and saved for all sectors.")


Generating SHAP explainability for Agriculture sector...
SHAP values saved for Agriculture in shap_values_Agriculture.pkl

Generating SHAP explainability for Industrial sector...
SHAP values saved for Industrial in shap_values_Industrial.pkl

Generating SHAP explainability for IT sector...
SHAP values saved for IT in shap_values_IT.pkl

Generating SHAP explainability for Manufacturing sector...
SHAP values saved for Manufacturing in shap_values_Manufacturing.pkl

Generating SHAP explainability for Residential sector...
SHAP values saved for Residential in shap_values_Residential.pkl

Generating SHAP explainability for Transport sector...
SHAP values saved for Transport in shap_values_Transport.pkl

Generating SHAP explainability for Power sector...
SHAP values saved for Power in shap_values_Power.pkl

SHAP explainability generated and saved for all sectors.


In [8]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 132.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

In [9]:

!pip install streamlit pyngrok --quiet

In [10]:
pip install streamlit pandas numpy joblib xgboost shap

In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import shap
import xgboost as xgb
import matplotlib.pyplot as plt

# List of sector-wise models and scalers
dataset_files = {
    "Agriculture": "agriculture_modified_data.csv",
    "Industrial": "industrial_modified_data.csv",
    "IT": "it_modified_data.csv",
    "Manufacturing": "manufacturing_modified_data.csv",
    "Residential": "residential_modified_data.csv",
    "Transport": "transport_modified_data.csv",
    "Power": "power_modified_data.csv"
}

st.title("Carbon Footprint Prediction App")
sector = st.selectbox("Select a sector:", list(dataset_files.keys()))

if sector:
    # Load trained XGBoost model and scalers
    xgb_model = joblib.load(f"xgb_model_{sector}.pkl")
    scaler_X = joblib.load(f"scaler_{sector}.pkl")
    scaler_y = joblib.load(f"scaler_y_{sector}.pkl")
    df = pd.read_csv(dataset_files[sector])
    sector_features = [col for col in df.columns if col != "emissions_tons"]

    st.subheader(f"Input features for {sector}:")
    user_input = {}
    for feature in sector_features:
        user_input[feature] = st.number_input(f"{feature}", min_value=0.0, format="%.2f")

    if st.button("Predict Carbon Footprint"):
        input_df = pd.DataFrame([user_input])
        X_scaled = scaler_X.transform(input_df)
        prediction = xgb_model.predict(X_scaled)
        predicted_emissions = scaler_y.inverse_transform(prediction.reshape(-1, 1))[0][0]
        st.success(f"Predicted Carbon Footprint: {predicted_emissions:.2f} tons")

        # SHAP explainability
        explainer = shap.Explainer(xgb_model)
        shap_values = explainer(X_scaled)

        st.subheader("Feature Contribution (SHAP Values)")
        shap_fig, ax = plt.subplots()
        shap.summary_plot(shap_values, X_scaled, feature_names=sector_features, show=False)
        st.pyplot(shap_fig)


Writing app.py


In [ ]:
import joblib

# List of sector-wise model files
sectors = ["Agriculture", "Industrial", "IT", "Manufacturing", "Residential", "Transport", "Power"]

for sector in sectors:
    try:
        # Load trained models
        xgb_model = joblib.load(f"xgb_model_{sector}.pkl")
        scaler_X = joblib.load(f"scaler_{sector}.pkl")
        scaler_y = joblib.load(f"scaler_y_{sector}.pkl")

        # Save models and scalers
        joblib.dump(xgb_model, f"xgb_model_{sector}.pkl")
        joblib.dump(scaler_X, f"scaler_X_{sector}.pkl")
        joblib.dump(scaler_y, f"scaler_y_{sector}.pkl")

        print(f"Saved models and scalers for {sector}")

    except Exception as e:
        print(f"Error processing {sector}: {e}")

In [22]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

# Define available sectors
sectors = ["Agriculture", "Industrial", "IT", "Manufacturing", "Residential", "Transport", "Power"]

st.title("Carbon Footprint Prediction App")
st.sidebar.header("User Input")

# Select sector
sector = st.sidebar.selectbox("Select Sector", sectors)

# Load necessary models and scalers
xgb_model = joblib.load(f"xgb_model_{sector}.pkl")
scaler_X = joblib.load(f"scaler_X_{sector}.pkl")
scaler_y = joblib.load(f"scaler_y_{sector}.pkl")
shap_values = joblib.load(f"shap_values_{sector}.pkl")

# Load sector-specific features
feature_names = scaler_X.feature_names_in_
user_input = {}

for feature in feature_names:
    user_input[feature] = st.sidebar.number_input(f"{feature}", value=0.0)

# Convert user input to dataframe
input_df = pd.DataFrame([user_input])
scaled_input = scaler_X.transform(input_df)

# Predict emissions
prediction = xgb_model.predict(scaled_input)
predicted_emissions = scaler_y.inverse_transform(prediction.reshape(-1, 1))[0][0]

st.write(f"### Predicted Carbon Footprint for {sector} Sector: {predicted_emissions:.2f} tons")

# SHAP Explanation
st.write("## SHAP Explainability")
explainer = shap.Explainer(xgb_model)
shap_values = explainer(scaled_input)
fig, ax = plt.subplots()
shap.waterfall_plot(shap.Explanation(values=shap_values.values[0], base_values=shap_values.base_values[0], feature_names=feature_names), max_display=10)
st.pyplot(fig)


Overwriting app.py


In [20]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

# Define available sectors
sectors = ["Agriculture", "Industrial", "IT", "Manufacturing", "Residential", "Transport", "Power"]

st.title("Carbon Footprint Prediction & Reduction Strategies")
st.sidebar.header("User Input")

# Select sector
sector = st.sidebar.selectbox("Select Sector", sectors)

# Load models and scalers
xgb_model = joblib.load(f"xgb_model_{sector}.pkl")
scaler_X = joblib.load(f"scaler_{sector}.pkl")
scaler_y = joblib.load(f"scaler_y_{sector}.pkl")
shap_values = joblib.load(f"shap_values_{sector}.pkl")

# Get feature names from trained scaler
feature_names = scaler_X.feature_names_in_

# User Input Features
user_input = {}
st.sidebar.write("### Adjust Input Features:")
for feature in feature_names:
    user_input[feature] = st.sidebar.number_input(feature, value=None, placeholder="Enter value")

# Predict Button
if st.sidebar.button("Predict Carbon Footprint"):
    if None in user_input.values():
        st.warning("Please enter values for all input features.")
    else:
        # Convert user input to dataframe
        input_df = pd.DataFrame([user_input])

        # Scale input features
        scaled_input = scaler_X.transform(input_df)

        # Predict emissions
        prediction = xgb_model.predict(scaled_input)
        predicted_emissions = scaler_y.inverse_transform(prediction.reshape(-1, 1))[0][0]

        st.write(f"## Predicted Carbon Footprint for {sector}: {predicted_emissions:.2f} tons")

        # SHAP Explainability
        st.write("## SHAP Explainability")
        explainer = shap.Explainer(xgb_model)
        shap_values = explainer(scaled_input)

        fig, ax = plt.subplots()
        shap.waterfall_plot(
            shap.Explanation(values=shap_values.values[0], base_values=shap_values.base_values[0], feature_names=feature_names),
            max_display=10
        )
        st.pyplot(fig)

        # Carbon Footprint Reduction Strategies
        st.write("## Reduction Strategies")

        recommendations = {
            "Agriculture": "Optimize fertilizer usage, adopt precision irrigation, and improve livestock feed efficiency.",
            "Industrial": "Upgrade to energy-efficient machinery, use recycled materials, and minimize process waste.",
            "IT": "Shift to energy-efficient data centers, optimize cooling systems, and prioritize cloud energy savings.",
            "Manufacturing": "Enhance machine efficiency, reduce material waste, and implement smart energy monitoring.",
            "Residential": "Adopt energy-efficient appliances, use solar power, and reduce water & gas consumption.",
            "Transport": "Switch to EVs/hybrids, improve fuel efficiency, and optimize travel routes.",
            "Power": "Increase renewable energy usage, improve grid efficiency, and reduce coal dependency."
        }

        st.write(f"**{recommendations[sector]}**")

        st.write("### Suggested Actions:")
        for feature, value in user_input.items():
            st.write(f"➡ Optimize {feature} to lower emissions.")

        st.write("Implementing these strategies can help reduce your carbon footprint!")

Overwriting app.py


In [15]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
import google.generativeai as genai  # Import Gemini chatbot API

# Set up Gemini API (Replace "YOUR_GEMINI_API_KEY" with your actual API key)
genai.configure(api_key="YOUR_GEMINI_API_KEY")
chatbot = genai.GenerativeModel("gemini-pro")

# Define available sectors
sectors = ["Agriculture", "Industrial", "IT", "Manufacturing", "Residential", "Transport", "Power"]

# Feature thresholds for reduction suggestions
feature_thresholds = {
    "Agriculture": {
        "energy_usage_mwh": 50,
        "crop_yield_tons": 100,
        "fertilizer_use_kg": 200,
        "livestock_count": 500,
        "irrigation_water_liters": 100000,
        "land_area_hectares": 1000,
    },
    "Industrial": {
        "energy_usage_mwh": 500,
        "raw_material_consumption_tons": 1000,
        "machine_operating_hours": 2000,
        "industrial_waste_generated_tons": 100,
        "factory_size_sq_m": 5000,
        "carbon_capture_tech_usage": 50,
    },
    "IT": {
        "server_energy_usage_mwh": 200,
        "data_center_size_sq_m": 1000,
        "electronic_waste_generated_kg": 500,
    },
    "Manufacturing": {
        "energy_usage_mwh": 600,
        "production_output_tons": 5000,
        "industrial_waste_generated_tons": 150,
        "water_usage_liters": 500000,
    },
    "Residential": {
        "electricity_usage_kwh": 1000,
        "gas_consumption_cubic_m": 500,
        "waste_generated_kg": 300,
    },
    "Transport": {
        "fuel_consumption_liters": 10000,
        "distance_traveled_km": 50000,
        "fleet_size": 100,
    },
    "Power": {
        "coal_usage_tons": 500,
        "renewable_energy_percentage": 20,
        "emissions_intensity_kg_per_mwh": 300,
    },
}

st.title("Carbon Footprint Prediction & Reduction Strategies")
st.sidebar.header("User Input")

# Select sector
sector = st.sidebar.selectbox("Select Sector", sectors)

# Load necessary models and scalers
xgb_model = joblib.load(f"xgb_model_{sector}.pkl")
scaler_X = joblib.load(f"scaler_X_{sector}.pkl")
scaler_y = joblib.load(f"scaler_y_{sector}.pkl")
shap_values = joblib.load(f"shap_values_{sector}.pkl")

# Load sector-specific features
feature_names = scaler_X.feature_names_in_
user_input = {}
user_provided_input = False  # Track if user has provided any input

for feature in feature_names:
    user_value = st.sidebar.number_input(f"{feature}", value=0.0)
    user_input[feature] = user_value
    if user_value != 0.0:
        user_provided_input = True  # Mark as true if any input is given

# Convert user input to dataframe
input_df = pd.DataFrame([user_input])

# Ensure prediction only runs when user provides input
if user_provided_input:
    scaled_input = scaler_X.transform(input_df)

    # Predict emissions
    prediction = xgb_model.predict(scaled_input)
    predicted_emissions = scaler_y.inverse_transform(prediction.reshape(-1, 1))[0][0]

    st.write(f"### Predicted Carbon Footprint for {sector} Sector: {predicted_emissions:.2f} tons")

    # Reduction Strategies
    st.write("## Reduction Strategies")
    suggested_actions = []
    thresholds = feature_thresholds.get(sector, {})

    for feature, value in user_input.items():
        if feature in thresholds and value > thresholds[feature]:
            suggested_actions.append(f"➡ Reduce {feature} (current: {value}, recommended: ≤ {thresholds[feature]})")
            suggested_actions.append("🔹 Possible Actions:")
            suggested_actions.append("   - Use energy-efficient equipment")
            suggested_actions.append("   - Optimize production schedules")
            suggested_actions.append("   - Implement waste reduction techniques")
            suggested_actions.append("   - Utilize renewable energy sources")

    if suggested_actions:
        st.write("### Suggested Actions:")
        for action in suggested_actions:
            st.write(action)
    else:
        st.write("✅ Your inputs are within optimal limits!")

    # SHAP Explanation
    st.write("## SHAP Explainability")
    explainer = shap.Explainer(xgb_model)
    shap_values = explainer(scaled_input)
    fig, ax = plt.subplots()
    shap.waterfall_plot(shap.Explanation(values=shap_values.values[0], base_values=shap_values.base_values[0], feature_names=feature_names), max_display=10)
    st.pyplot(fig)

    # Gemini Chatbot for Further Advice
    st.write("## Ask Gemini AI for More Insights")
    user_query = st.text_input("Ask for additional carbon footprint reduction strategies:")

    if st.button("Ask Gemini"):
        response = chatbot.generate_content([user_query])
        st.write("### Gemini AI Response:")
        st.write(response.text)

else:
    st.write("🔹 Please provide input values to generate predictions.")

Overwriting app.py


In [21]:
!streamlit run app.py & npx localtunnel --port 8501





⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.36.66:8501

your url is: https://wet-numbers-do.loca.lt
  Stopping...
^C


In [12]:
!streamlit run app.py & sleep 5




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.36.66:8501

  Stopping...
